To-do:
1. Т.к в нашем распоряжении есть расчет скора по нескольким статусам, необходимо объединить 2 показателя в единую метрику оценки. Разработайте функцию объеденения, например рачетом среднего арифметического показателей по статусам. На вход функция должна принимать pandas dataframe с расчетом скора по статуасам, на выходе должен быть pandas dataframe со схемой {"assignee_id":assignne_id,"score_value":score_total}
2. В примере представлен скоринг только по одному агенту поддержки. Необходимо разработать программный код для скоринга по статусам всех имеющихся пользователям, результаты расчета сохранить в один pandas dataframe. Точно так же сформировать итоговый скор по каждому агенту поддержки и сохранить во второй pandas dataframe. В итоге у вас должно получиться 2 pandas dataframe, в одном результаты скоринга агентов поддержки в разрезе статуса (closed, solved), во втором результаты итогово скоринга агентов поддержки
3. Записать результаты расчетов в соответствующие таблицы базы данных BigQuery. В примере представлена функция для записи данных скоринга по статусам, сделайте это функцию универсальной для записи обоих наборов данных в соответствующие таблицы
4. Подготовка продакшен программного кода. Организуйте функции расчета скоринга в отдельный lib_main.py . Ораганизуйте __main__.py файл, импортируйте в него сожержимое файла lib_main.py, разработайте программный код извлечения сырых данных из базы данныз BigQuery, расчет скоринга по статусам и итогового скоринга, сделайте запись результатов работы алгоритма скоринга в соответствующие таблицы базы данных BigQuery. В итоге при запуске __main__.py файла должен полностью выполняться весь алгоритм скоринга без дополнительного вмешательства.
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
5. В качестве гипотезы попробуйте добавить дополнительный разрез данных - канал в котором пришло обращение и расчитать загрузку по статусам и по каналам одновременно
6. В качестве гипотезы попробуйте использовать более робастные центральные меры данных и меры вариабильности, попробуйте иные способы формирования доверительного интервала
7. Напишите документацию к вашему алгоритму. Офишите математические концепции кратко и понятно, без глубоких выкладок, опишите источники данных, опишите схемы результирующих таблиц таким образом, чтобы было понятно любому человеку (почти)

In [83]:
from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt

In [84]:
CREDENTIALS = service_account.Credentials.from_service_account_info({
#   INSERT CREDENTIALS HERE
})

In [148]:
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
SqlQuery - string, sql query for BigQeury database
[example]
Input: Credentials = credentials_object
       SqlQuery = 'select * from dataset_name.table_name'
Output: id	    created_at	        updated_at	        type	  subject   description	                                                                                          status	requester_id	submitter_id   assignee_id	 id_project	 id_invoice	channel	country	 manual_category  auto_category	 subcategory   feedback_score	feedback_comment
	    2520211	2018-02-05 08:59:15	2018-02-23 13:05:39	question	        Credit card payments not working from website.	I have been trying since Thursday 1st of Jan t...	  closed	360790164527	360790164527   20890258907	 21190	     316520736	other	za	     None	          None	         None	       unoffered	    None
        2740781	2018-08-17 01:48:04	2018-09-15 11:00:15	question	        Re: error showed during paid subscription on t...	__________________________________\nType your ... closed	365082895633	951579756	   360133124587	 15174	     367443669	email	za	     None	          None	         None	       offered	        None
"""
def getFreshData(Credentials,ProjectId):
    bigquery_sql = " ".join(["SELECT id, DATE(CAST(created_at AS DATETIME)) AS created, DATE(CAST(updated_at AS DATETIME)) AS updated, status, assignee_id, channel",
                             "FROM `xsolla_summer_school.customer_support`",
                             "WHERE status IN ('closed','solved')",
                             "ORDER BY updated_at"])

    dataframe = pandas_gbq.read_gbq(bigquery_sql,project_id=ProjectId, credentials=Credentials, dialect="standard")

    return dataframe


"""[summary]
Function for scoring workload by statuses (In Progress and Done) for one employee, NumOfAllDays = 63, NumOfIntervalDays = 7
[description]
Data - pandas dataframe object, with hist data for customer support agent
NumOfAllDays - integer, number of days for all hist data
NumOfIntervalDays - integer, number of days for weekly calculating interval
[example]
Input: Data = id	   created	   updated	   status	assignee_id
              2936149  2018-12-31  2019-01-30  closed	26979706288
              2936171  2018-12-31  2019-01-30  closed	26979706288
       NumOfAllDays = 63
       NumOfIntervalDays = 7
Output: assignee_id	 status	 count_last_period	count_mean_calc_period	count_sem_calc_period	score_value
        12604869947	 closed	 196	            196.62	                9.43	                1
        12604869947	 solved	 0	                0.00	                0.00	                0    
"""
def workloadScoringByStatusesAndChannels(Data,NumOfAllDays,NumOfIntervalDays):
    assignee_id = np.unique(Data.assignee_id)
    assignee_id = assignee_id[0]
    
    #splitting by status
    statuses = np.unique(Data.status)
    channels = np.unique(Data.channel)
    assignee_id_list = []
    status_list = []
    channel_list = []
    avg_num_of_task_per_week_list = []
    ste_list = []
    num_tasks_per_current_week_list = []
    score_for_status_list = []
    
    for channel in channels:        
        for status in statuses:
            
            dataframe_status = Data[(Data.status == str(status)) & (Data.channel == str(channel))][:]
        
            #time borders params
            curr_date = dt.datetime.strptime(str('2017-04-01'),'%Y-%m-%d')
            curr_date = curr_date.date()
            delta = dt.timedelta(days=NumOfAllDays)
            first_date = curr_date-delta
        
            #time interval params
            delta_interval = dt.timedelta(days=NumOfIntervalDays)
            first_interval = first_date+delta_interval
                
            num_of_intervals = int(NumOfAllDays/NumOfIntervalDays)
            num_tasks_per_week = []
            for i in range(0,num_of_intervals):
                interval = dataframe_status[(dataframe_status.updated >= str(first_date)) & (dataframe_status.updated <= str(first_interval))][:]
                first_date = first_date + delta_interval
                first_interval = first_interval + delta_interval
        
                if i != (num_of_intervals-1):        
                    num_of_tasks = len(np.unique(interval['id']))
                    num_tasks_per_week.append(num_of_tasks) #history number of tasks
                else:
                    num_tasks_per_current_week = len(np.unique(interval['id'])) #currently number of tasks
            
            avg_num_of_task_per_week = round(np.mean(num_tasks_per_week),2)
    
            #squared deviations
            x_values = []
            for num in num_tasks_per_week:
                x = round((num - avg_num_of_task_per_week)**2,2)
                x_values.append(x)
    
            #data sampling statistics
            x_sum = round(sum(x_values),2) #sum of squared deviations
            dispersion = round(x_sum/(num_of_intervals-1),2) #dispersion
            std = round(mt.sqrt(dispersion),2) #standart deviation for sample
            ste = round(std/mt.sqrt(num_of_intervals),2) #standart error for sample
    
            #confidence interval
            left_border = int(avg_num_of_task_per_week - 2 * ste)
            right_border = int(avg_num_of_task_per_week + 2 * ste)
    
            #workload scoring for status
            score_for_status = workloadScoreStatuses(left_border,right_border,num_tasks_per_current_week)        
            assignee_id_list.append(assignee_id)
            status_list.append(status)
            channel_list.append(channel)
            avg_num_of_task_per_week_list.append(avg_num_of_task_per_week)
            ste_list.append(ste)
            num_tasks_per_current_week_list.append(num_tasks_per_current_week)
            score_for_status_list.append(score_for_status)
        
    score_data = {"assignee_id":assignee_id_list,"channel": channel_list, "status":status_list,
                  "count_last_period":num_tasks_per_current_week_list,"count_mean_calc_period":avg_num_of_task_per_week_list,"count_sem_calc_period":ste_list,
                  "score_value":score_for_status_list}
    scores = pd.DataFrame(data=score_data)

    return scores


"""[summary]
Function for scoring workload for current status
[description]
LeftBoard - float, left boarder for confidence interval
RightBoard - float right boarder for confidence interval
CurrentNumOfTasks - integer, number of customer support agent tasks for current interval (7 days)
[example]
Input: LeftBoard = 187
       RightBoard = 206
       CurrentNumOfTasks = 196
Output: 1
"""
def workloadScoreStatuses(LeftBoard,RightBoard,CurrentNumOfTasks):
    if (LeftBoard == 0) & (CurrentNumOfTasks == 0) & (RightBoard == 0):
        score = 0
    elif (CurrentNumOfTasks >= 0) & (CurrentNumOfTasks < LeftBoard):
        score = 0
    elif (CurrentNumOfTasks >= LeftBoard) & (CurrentNumOfTasks <= RightBoard):
        score = 1
    else:
        score = 2
    
    return score


"""[summary]
Function for inserting data to BigQuery database
[description]
InsertDataFrame - pandas dtaframe object, with score result data by statuses
ProjectId - string, name of project in google cloud platform 
DatasetId - string, name of dataset in bigquery for raw data
TableId - string, name of table for raw data
[example]
Input: InsertDataFrame = assignee_id	status	count_last_period	count_mean_calc_period	count_sem_calc_period	score_value
                         11527290367	closed	163	                140.38	                12.4	                2
                         11527290367	solved	0	                0.00	                0.0 	                0
       ProjectId = 'test-gcp-project'
       DatasetId = 'test_dataset'
       TableId = 'test_table'
"""
def insertScoreResultData(InsertDataFrame,ProjectId,DatasetId,TableId):
    destination_table = f"{DatasetId}.{TableId}"
    
#     res_df = pd.DataFrame()
#     res_df['assignee_id'] = InsertDataFrame['assignee_id'].astype('int')
#     res_df['status'] = InsertDataFrame['status'].astype('str')
#     res_df['count_last_period'] = InsertDataFrame['count_last_period'].astype('int')
#     res_df['count_mean_calc_period'] = InsertDataFrame['count_mean_calc_period'].astype('float')
#     res_df['count_sem_calc_period'] = InsertDataFrame['count_sem_calc_period'].astype('float')
#     res_df['score_value'] = InsertDataFrame['score_value'].astype('int')
#     res_df['developer'] = 'evgeniy.zorin'
#     res_df['developer'] = res_df['developer'].astype('str')
    res_df = InsertDataFrame
    res_df['developer'] = 'evgeniy.zorin'
    res_df['developer'] = res_df['developer'].astype('str')
    pandas_gbq.to_gbq(res_df, destination_table=destination_table, project_id=ProjectId, if_exists='append')  
    
    
def get_total_score(data):
    total_score = data[data['count_last_period'] > 0]['score_value'].median()    
    score_data = {'assignee_id': data['assignee_id'].iloc[0], 'score_value': [total_score]}    
    return pd.DataFrame(score_data).fillna(0)

In [86]:
DataFrame = getFreshData(CREDENTIALS,'findcsystem')
DataFrame.head(10)

Downloading: 100%|█████████████████████████████████████████████████████████| 350461/350461 [00:42<00:00, 8332.17rows/s]


,id,created,updated,status,assignee_id,channel
0,2140147,2017-01-01,2017-01-01,closed,4225872478,help_widget
1,2140305,2017-01-01,2017-01-01,closed,379332361,help_widget
2,2140360,2017-01-01,2017-01-01,closed,12604869947,help_widget
3,2140375,2017-01-01,2017-01-01,closed,12604869947,help_widget
4,2140359,2017-01-01,2017-01-01,closed,12604869947,help_widget
5,2140841,2017-01-01,2017-01-01,closed,379332361,help_widget
6,2141392,2017-01-02,2017-01-02,closed,14318037588,help_widget
7,2142443,2017-01-02,2017-01-02,closed,774796347,help_widget
8,2142384,2017-01-02,2017-01-02,closed,12604869947,help_widget
9,2142389,2017-01-02,2017-01-02,closed,12604869947,help_widget


In [95]:
DataFrame = DataFrame.fillna('None')

In [96]:
#list of unique support agents
users = DataFrame['assignee_id'].unique()
users

array([  4225872478,    379332361,  12604869947,  14318037588,
          774796347,   1430684637,    291643586,   8061106928,
          867359386,  10574264847,   2329316498,    288517962,
        16913504828,  15294499287,  10735982687,    291458441,
         1162180708,  18168708607,  11527290367,  18251419728,
        19185592868,  19949080948,  20890258907,  21627434567,
        21703602368,  20966194228,  21509255328,  22415990187,
        25016708788,    291235568,  24524589027,  25314245267,
        25314880867,  25314864667,  25960267408,  25872102447,
        26979706288,  27220108387,    123193832,  27654508988,
       361576272808, 361596485307,    502786743, 361576678608,
       360133124587, 363719971907, 363719974507, 364566660327,
       364726003288, 365273409867, 364622202673, 365657486013,
       365658390953], dtype=int64)

In [145]:
test_user = DataFrame[DataFrame.assignee_id == 22415990187][:]
test_user.reset_index(inplace=True, drop=True)
test_user.head(10)

,id,created,updated,status,assignee_id,channel
0,2278673,2017-05-23,2017-05-23,closed,22415990187,help_widget
1,2285516,2017-05-30,2017-05-30,closed,22415990187,help_widget
2,2285652,2017-05-30,2017-05-31,closed,22415990187,help_widget
3,2272549,2017-05-16,2017-06-13,closed,22415990187,email
4,2272569,2017-05-16,2017-06-13,closed,22415990187,help_widget
5,2272650,2017-05-16,2017-06-14,closed,22415990187,email
6,2271721,2017-05-15,2017-06-15,closed,22415990187,help_widget
7,2273402,2017-05-17,2017-06-15,closed,22415990187,help_widget
8,2273533,2017-05-17,2017-06-15,closed,22415990187,help_widget
9,2272566,2017-05-16,2017-06-16,closed,22415990187,help_widget


In [146]:
test_user.shape

(19578, 6)

In [157]:
test_result = workloadScoringByStatusesAndChannels(test_user,63,7)
test_result

,assignee_id,channel,status,count_last_period,count_mean_calc_period,count_sem_calc_period,score_value
0,22415990187,None,closed,0,0.0,0.0,0
1,22415990187,None,solved,0,0.0,0.0,0
2,22415990187,call,closed,0,0.0,0.0,0
3,22415990187,call,solved,0,0.0,0.0,0
4,22415990187,chat,closed,0,0.0,0.0,0
5,22415990187,chat,solved,0,0.0,0.0,0
6,22415990187,email,closed,0,0.0,0.0,0
7,22415990187,email,solved,0,0.0,0.0,0
8,22415990187,facebook,closed,0,0.0,0.0,0
9,22415990187,facebook,solved,0,0.0,0.0,0


In [149]:
total_scores_table = pd.DataFrame()
status_scores_table = pd.DataFrame()

for user in users:
    workloadScores = workloadScoringByStatusesAndChannels(DataFrame[DataFrame.assignee_id == user][:],63,7)
    status_scores_table = pd.concat([status_scores_table, workloadScores]) 
    total_scores_table = pd.concat([total_scores_table, get_total_score(workloadScores)])

In [150]:
status_scores_table.reset_index(inplace=True, drop=True)
status_scores_table.head()

,assignee_id,channel,status,count_last_period,count_mean_calc_period,count_sem_calc_period,score_value
0,4225872478,None,closed,0,0.00,0.00,0
1,4225872478,None,solved,0,0.00,0.00,0
2,4225872478,call,closed,9,29.38,4.92,0
3,4225872478,call,solved,0,0.00,0.00,0
4,4225872478,chat,closed,22,143.75,26.80,0


In [151]:
total_scores_table.reset_index(inplace=True, drop=True)
total_scores_table

,assignee_id,score_value
0,4225872478,0.0
1,379332361,0.5
2,12604869947,1.0
3,14318037588,0.0
4,774796347,0.0
5,1430684637,0.0
6,291643586,0.0
7,8061106928,0.5
8,867359386,1.0
9,10574264847,1.0


In [152]:
status_scores_table['assignee_id'] = status_scores_table['assignee_id'].astype('int64')
status_scores_table['status'] = status_scores_table['status'].astype('str')
status_scores_table['channel'] = status_scores_table['channel'].astype('str')
status_scores_table['count_last_period'] = status_scores_table['count_last_period'].astype('int32')
status_scores_table['count_mean_calc_period'] = status_scores_table['count_mean_calc_period'].astype('float')
status_scores_table['count_sem_calc_period'] = status_scores_table['count_sem_calc_period'].astype('float')
status_scores_table['score_value'] = status_scores_table['score_value'].astype('int32') 

In [153]:
insertScoreResultData(status_scores_table,'findcsystem','xsolla_summer_school','score_result_status_channel')

1it [00:07,  7.23s/it]


In [154]:
total_scores_table['assignee_id'] = total_scores_table['assignee_id'].astype('int64')

In [155]:
insertScoreResultData(total_scores_table,'findcsystem','xsolla_summer_school','score_result_total')

1it [00:05,  5.41s/it]
